# What is AeroTools

AeroTools is a collection of tools, mostly written in Python, for reading, analyzing, visualizing and evaluating model and observational data. The tools are mean to be easy and quick to use, while at the same time providing thorough and easy to understand results. 

While AeroTools can be used in many different ways, the main output of AeroTools is files that can be hosted on the beautiful [Aeroval webpage](https://aeroval.met.no/evaluation/?project=emep&exp_name=2023-reporting). This webpage has been used by the air pollution group at FoU-KL for a couple of year, with one use being the evaluation used in the yearly EMEP report. 

## Quick Tour of AeroTools

In this tutorial we will look at the main components of AeroTools: PyAerocom, PyAeroval and Aeroval. This is just a quick overview of AeroTools. Each tool has their own tutorials for those how wants to dive deeper into them

- PyAerocom: The biggest part of AeroTools. Responsible for reading data, colocating the data as well as pre- and post-processing the data. The final product of PyAerocom is a ColocatedData object
- PyAeroval: The part of AeroTools most users work with. Calculates timeseries, statistics, etc, for use in the Aeroval webpage.
- Aeroval: The beautiful webpage used to display the results from PyAeroval

As of now, both PyAerocom and PyAeroval are part of the PyAerocom library. Aeroval has it's own repository, which is internal to MET. We will in a later tutorial see how to upload your own results to a development version of this webpage.

Below is a diagram of said components

![AeroTools Components](aeroTools_components.png)

There are other (planned) tools in AeroTools, which are not part of the main workflow, but which can do smaller, specific tasks. On such tool is the [NorESM-Aerocom-Converter](https://github.com/metno/noresm-aerocom-converter)(NAC), which we will use to convert NorESM model data to Aerocom data.

### PyAerocom

PyAerocom is the biggest part of the AeroTools, and is the core on which the rest is build upon. PyAerocom is used as a normal python library, by calling of classes, methods, etc, contrary to PyAeroval (as we shall see below). 

PyAerocom is responsible for 

- Reading data
    - Model data is read from gridded data (mostly netCDF). If the data file(s) is Aerocom compliant and the variables you want to use are registered in PyAerocom, then PyAerocom should be able to read the file(s).
    - Observations can be both gridded data (e.g. satellite data) and ungridded data (most observational networks). Ungridded data is generally difficult to read, as each observational network has its own file format. PyAerocom has built in several readers for different observational networks. At the start of 2024, PyAerocom has started to use [Pyaro](https://github.com/metno/pyaro), which is a separate MET project meant to make it easier to write your own ungridded data reader, without having to mess with the source code of PyAerocom. We will in another tutorial see how to use Pyaro when reading data (but not how to make a reader, for that see the [Pyaro documentation](https://pyaro.readthedocs.io/en/latest/).
    - Ungridded data can be seen as a collection of StationData objects. These are objects holding timeseries of observations, as well as metadata, for separate measurement stations.
    - The classes for reading data will return either a *GriddedData* or a *UngriddedData* objects (as well as *StationData* objects coming from the UngriddedData objects. These can be used on there own, without the need for colocation.
- Pre- and post-processing can be done on the data before and after reading, as well as before and after colocation. This include excluding and filtering measurement stations, filtering measurements, as well as calculate timeseries for coarser measurement frequencies (e.g. going from daily to monthly timeseries)
- Colocation is the final step of PyAerocom, and consists of matching model data with observations
    - The most common way of colocation in Gridded-Ungridded colocation, where gridded model data is matched with ungridded observational data. In this case gridded model data is interpolated to the position (and time) of the measurements done at a station, so that the model and observation can be compared.
    - The newest feature of PyAerocom is the ability to colocate vertical profiles.
 
As mentioned above, the final result of PyAerocom is a *ColocatedData* object. This can used on its own, e.g. for plotting of user defined analysis.

![Parts of Pyaerocom](pyaerocom_coldata.png)

### PyAeroval

PyAeroval is a tool made for analyzing model data and observation using PyAerocom. PyAeroval will use PyAerocom to read and colocated the data, and then do a evaluation of the given data. This is meant to be viewed on the Aeroval webpage, which we will discuss below.

PyAeroval is how most users will use AeroTools. When using PyAeroval you do not have to touch PyAerocom. Instead you will have to make a configuration file which defines the models, observations, chemical species, filters, etc you want to use. Writing these config files might seem daunting at first, but when you first have a config file you are happy with, you will mostly copy and reuse this basic structure for most of your config files. In later tutorials you will be gently introduced to how to setup such a config file.

Then PyAeroval is done running you will end up with a large amount of *json* files, containing the results. While it is possible to use these to make your own plots, they are meant to be used with Aeroval

### Aeroval

Aeroval can be seen as the main selling point of AeroTools. This is a hub used to watch and share you evaluation. 

Aeroval is divided into projects and experiments. Projects are collections of experiments. One example of a project is EMEP, with the different yearly reports being experiments. 

The evaluations are done for each model, observation and chemical species. A long with time series, a range of [statistical quantities](https://aeroval.met.no/infos/?project=emep&exp_name=2023-reporting) are calculated and show.

Aeroval is meant to be shared with other people, but plots can be extraceted and downloaded from Aeroval as well, e.g. for the use in articles.

## Using the AeroTools Module

In this last section we will show how to activate AeroTools, with all the tools needed for the rest of the workshop. It is possible to install all AeroTools locally using python and pip, but a goal of AeroTools is that it should be easily accessible on PPI. I will thus only show how to use it on PPI. 

AeroTools is already installed on PPI as a module, we therefore only need to load said module

In [ ]:
%%bash
module load /modules/MET/rhel8/user-modules/fou-kl/aerotools/aerotools

And we are ready to use AeroTools

In [1]:
%%bash
pya --help

                                                                                
 Usage: pya [OPTIONS] COMMAND [ARGS]...                                         
                                                                                
 Pyaerocom Command Line Interface                                               
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --version             -V                                                     │
│ --install-completion            Install completion for the current shell.    │
│ --show-completion               Show completion for the current shell, to    │
│                                 copy it or customize the installation.       │
│ --help                          Show this message and exit.                  │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ────────────────

### <a name="condamodules"> Anaconda Modules </a>

One feature of the modules that is worth noting, is that there is a difference between the modules `aerotools` and `aerotools.conda`. As might come as a surprise, the latter is based on Anaconda, while the former is not. From a user perspective, the difference is that the latter will become your main python install, former will become a separate python install, working besides your default python. This means that you need to call python with the command `pya_python`.